# Portfolio Optimization Analysis

## Introduction
In this analysis, we explore portfolio optimization with a focus on 40 stocks that echo the S&P 500's sector distribution. We'll evaluate their performance and optimize portfolios for maximum Sharpe ratio, return, and minimal risk. A comparative view will highlight shifts from the original S&P 500 sector distribution, offering insights into the balancing act between risk and return within optimized portfolios.

## Load and Analyze the Data
First, let's load the dataset, wich consists of selected 40 stocks representing the S&P 500 index's sector-wise weight distribution. We'll analyze the data to gain insights into the stocks and their respective sectors.